In [8]:
from webbot import Browser
from bs4 import BeautifulSoup
import time
import json  
import pprint


In [9]:
dataDict        = {} #initialize a dictionary to store the data we want
playerStateDict = {} #initialize a dictionary to store players current state
histDataDict    = {} #initializes a dictionary to store players historical data
fdrDataDict     = {} #initializes a dictionary to store the FDR details for players
global index
global historicalIndex
global currentPage
index = currentPage = fdrIndex =  historicalIndex = 1


In [10]:
webBrowser  = Browser(showWindow=True)
webBrowser.go_to('https://fantasy.premierleague.com/statistics')
time.sleep(5)

In [11]:
#Functions

#scraps the historical Data
def scrapeHistoricalData(webBrowserObj,scrapDataIndex,index,historicalIndex):
    webBrowserObj.click(tag='button',classname='ElementDialogButton__StyledElementDialogButton-sc-1vrzlgb-0',number = scrapDataIndex)
   
    #gets Player FDR score
    webBrowserObj.click('Fixtures')
    time.sleep(5)
    webContent = webBrowserObj.get_page_source()
    soup = BeautifulSoup(webContent)
    data = soup.find("table", { "class" : "fVnGhl" })
    scrapHistDataIndex = 1
    rowData = data.find_all('tr')
    global fdrIndex
    for row in rowData:
        if len(rowData) > scrapHistDataIndex:
            singleRow=rowData[scrapHistDataIndex].find_all('td')
            fdrDataDict[fdrIndex] = [index,singleRow[0].text, #date 
                               singleRow[1].text, #GW - game week 
                               singleRow[2].text, #opponent
                               singleRow[3].text #FDR             
                              ]
        scrapHistDataIndex += 1
        fdrIndex +=1
    webBrowser.click(tag='button',classname='rPLLF')
    time.sleep(5)
     
#moves the cursor to the next page for scrapping
def getDataForNextPage(webBrowserObj,index,currentPage,historicalIndex):
    webBrowserObj.go_to('https://fantasy.premierleague.com/statistics')
    time.sleep(10)
    counter = 1
    while currentPage > counter:
        if counter == 1:
            print("page - counter is: ",counter)
            webBrowserObj.click(tag='button',classname='kRIEWN')
        elif counter > 1:
            print("page - counter is: ",counter)
            webBrowserObj.click(tag='button',classname='kRIEWN', number = 2)
        counter += 1
        time.sleep(8)
    scrapData(webBrowserObj,index,currentPage,historicalIndex)

#main scrap function    
def scrapData(webBrowserObj,index,currentPage,historicalIndex):
    webContent = webBrowserObj.get_page_source()
    soup = BeautifulSoup(webContent)
    data = soup.find("table")
    scrapDataIndex = 1
    rowData = data.find_all('tr')
    for row in rowData:
        if len(rowData) > scrapDataIndex:# and scrapDataIndex < 4:
            #function gets historical Data
            scrapeHistoricalData(webBrowserObj,scrapDataIndex,index,historicalIndex)
            index += 1
            scrapDataIndex += 1
            historicalIndex += 1
        elif scrapDataIndex == len(rowData) and currentPage < 24:
            currentPage += 1
            print("current Page is: ",currentPage)
            getDataForNextPage(webBrowserObj,index,currentPage,historicalIndex)
            

In [12]:
getDataForNextPage(webBrowser,index,currentPage,historicalIndex)
#getDataForNextPage(webBrowser,58,2,470)
#getDataForNextPage(webBrowser,1,1,1485)

current Page is:  2
page - counter is:  1
current Page is:  3
page - counter is:  1
page - counter is:  2
current Page is:  4
page - counter is:  1
page - counter is:  2
page - counter is:  3
current Page is:  5
page - counter is:  1
page - counter is:  2
page - counter is:  3
page - counter is:  4
current Page is:  6
page - counter is:  1
page - counter is:  2
page - counter is:  3
page - counter is:  4
page - counter is:  5
current Page is:  7
page - counter is:  1
page - counter is:  2
page - counter is:  3
page - counter is:  4
page - counter is:  5
page - counter is:  6
current Page is:  8
page - counter is:  1
page - counter is:  2
page - counter is:  3
page - counter is:  4
page - counter is:  5
page - counter is:  6
page - counter is:  7
current Page is:  9
page - counter is:  1
page - counter is:  2
page - counter is:  3
page - counter is:  4
page - counter is:  5
page - counter is:  6
page - counter is:  7
page - counter is:  8
current Page is:  10
page - counter is:  1
page 

In [ ]:
#pprint.pprint(histDataDict) #prints the historical data summary for the first 2 players
pprint.pprint(fdrDataDict) 

In [9]:
with open("dumps/fdrDataDict.json", "w") as outfile: 
        json.dump(fdrDataDict, outfile)